This package is to run data modelling/forecast predictions based on historical data.

The sqlite database keeps a record of what stock data is on hand, when it begins, and when it ends.

When data is requested from the database, if it is not available in the database, execute a request to the API (and update the ticker table as well as the metadata table in the database)

In [1]:
import pandas as pd
from db_utilities import query, list_all_tables, drop_ticker_table, read_table, update_ticker_metadata
from setup_utilities import create_metadata_table, initial_data_pull, create_ticker_metadata
from stock_functions import get_close_price, get_ohlc
import datetime as dt 
import sqlite3 


# populate your universe of stocks


#known issues with this 'universe
bad_tickers = ['ABMD','ATVI','ALL','ABC','ANTM','BLL','BRK.B','BF.B','CERN','CTXS','DISCA','DISCK','DISH','DRE','RE','FB','FRC','FISV','FBHS','INFO','KSU','NLSN','NLOK','PBCT','PKI','SIVB','TWTR','VIAC','WLTW','XLNX']
data = pd.read_csv('data/constituents.csv')
data = data[~data['Symbol'].isin(bad_tickers)]
all_tickers = sorted(list(x for x in data['Symbol']))


drop_ticker_table('stock_metadata')
create_metadata_table()
for idx, row in data.iterrows():
    create_ticker_metadata(row['Symbol'], row['Name'], row['Sector'])
    
len(all_tickers)
for x in all_tickers:
    update_ticker_metadata(x)

C:\Users\polan\AppData\Local\Temp\ipykernel_17120\3010129004.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


dropped table stock_metadata


In [2]:
read_table('stock_metadata')


,name,industry,rowcount,last_updated,ticker,first_date,last_date
0,3M,Industrials,3046,2024-02-11,MMM,2012-01-03 00:00:00-05:00,2024-02-09 00:00:00-05:00
1,A. O. Smith,Industrials,3046,2024-02-11,AOS,2012-01-03 00:00:00-05:00,2024-02-09 00:00:00-05:00
2,Abbott Laboratories,Health Care,3046,2024-02-11,ABT,2012-01-03 00:00:00-05:00,2024-02-09 00:00:00-05:00
3,AbbVie,Health Care,2796,2024-02-11,ABBV,2013-01-02 00:00:00-05:00,2024-02-09 00:00:00-05:00
4,Accenture,Information Technology,3046,2024-02-11,ACN,2012-01-03 00:00:00-05:00,2024-02-09 00:00:00-05:00
...,...,...,...,...,...,...,...
470,Yum! Brands,Consumer Discretionary,3046,2024-02-11,YUM,2012-01-03 00:00:00-05:00,2024-02-09 00:00:00-05:00
471,Zebra Technologies,Information Technology,3046,2024-02-11,ZBRA,2012-01-03 00:00:00-05:00,2024-02-09 00:00:00-05:00
472,Zimmer Biomet,Health Care,3046,2024-02-11,ZBH,2012-01-03 00:00:00-05:00,2024-02-09 00:00:00-05:00
473,Zions Bancorp,Financials,3046,2024-02-11,ZION,2012-01-03 00:00:00-05:00,2024-02-09 00:00:00-05:00


In [4]:
# main update job

issues = []
# def update_all():
ref_table = read_table('stock_metadata')

for idx, row in ref_table.iterrows():
    print(f"updating metadata for {row['ticker']}", end='\t\r', flush=True)
    update_ticker_metadata(row['ticker'])
    if "".join(row['last_date'][:10].split('-')) < (dt.datetime.now().date()-dt.timedelta(days=4)).strftime('%Y%m%d'):
        print(f"\nupdating rows for row {row['ticker']}", end='\t\r', flush=True)
        # print(row['last_date'], row['last_date'] < dt.datetime.now().date().strftime('%Y%m%d'), dt.datetime.now().date().strftime('%Y%m%d'))
        
        new_data = get_ohlc(row['ticker'], start_date=dt.datetime.fromisoformat(row['last_date'])+dt.timedelta(days=1), end_date=dt.datetime.now().date())
        columns = ",".join([f"'{x}'" for x in new_data.columns])
        con = sqlite3.connect('stocks.db')
        with con:
            try:
                for local_row in new_data.iterrows():

                    values = ",".join([f"'{x}'" for x in local_row[1].values])
                    #print(f'INSERT INTO {table_name} ({columns}) VALUES ({values})')
                    con.execute(f"INSERT INTO {row['ticker']} ({columns}) VALUES ({values})")
                    con.commit()
            except BaseException as e:
                print(e)
                issues.append(row['ticker'])
                con.rollback()
                
        update_ticker_metadata(row['ticker'])
print(f"\nissues with {issues}")

updating metadata for ZTS																						
issues with []
